In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

<IPython.core.display.Javascript object>

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [3]:
# answer goes here
nba = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
nba.head()



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


<IPython.core.display.Javascript object>

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [4]:
# answer goes here
nba = nba[nba["GS"] >= 20]
nba = nba[nba["MP"] >= 10]
nba.head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,...,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,...,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4


<IPython.core.display.Javascript object>

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [5]:
# answer goes here
col = ["PTS", "TRB", "AST", "STL", "BLK", "TOV", "Age"]
nba = nba[col]
nba.head()

,PTS,TRB,AST,STL,BLK,TOV,Age
3,13.9,9.5,1.6,1.5,1.0,1.7,25
4,8.9,7.3,2.2,0.9,0.8,1.5,21
7,21.3,9.2,2.4,0.5,1.3,1.8,33
10,10.9,8.4,1.4,0.5,1.5,1.3,20
12,9.4,7.5,1.3,0.8,0.4,0.9,28


<IPython.core.display.Javascript object>

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [6]:
# answer goes here
scaler = StandardScaler()
scaled = scaler.fit_transform(nba)

<IPython.core.display.Javascript object>

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [16]:
# answer goes here
k_means3 = KMeans(n_clusters=3)
k_means3.fit(scaled)
print(
    pd.DataFrame(
        scaler.inverse_transform(k_means3.cluster_centers_), columns=nba.columns
    )
)

gmm3 = GaussianMixture(n_components=3)
gmm3.fit(scaled)
print(pd.DataFrame(scaler.inverse_transform(gmm3.means_), columns=nba.columns))

k_means7 = KMeans(n_clusters=7)
k_means7.fit(scaled)
print(
    pd.DataFrame(
        scaler.inverse_transform(k_means7.cluster_centers_), columns=nba.columns
    )
)

gmm7 = GaussianMixture(n_components=7)
gmm7.fit(scaled)
print(pd.DataFrame(scaler.inverse_transform(gmm7.means_), columns=nba.columns))


         PTS       TRB       AST       STL       BLK      TOV        Age
0  10.450455  3.956136  2.063864  0.806591  0.369318  1.22750  26.756818
1  13.724044  8.528962  1.912568  0.771038  1.231694  1.67377  25.896175
2  19.501657  5.289503  5.895028  1.350276  0.486188  2.78895  26.320442
         PTS       TRB       AST       STL       BLK       TOV        Age
0  10.198945  4.458926  1.581203  0.780490  0.425599  1.103803  26.742897
1  14.384442  8.477493  2.246641  0.773751  1.223151  1.821688  26.137352
2  15.991231  4.200315  4.825259  1.178664  0.374159  2.255369  26.350213
         PTS        TRB       AST       STL       BLK       TOV        Age
0  17.968000  10.674000  2.244000  0.960000  1.850000  2.182000  25.040000
1   7.447328   3.695420  1.264122  0.585496  0.405344  0.890076  24.114504
2   9.960959   4.074658  2.104110  0.747945  0.378082  1.162329  31.547945
3  13.800541   4.190811  2.762703  1.056216  0.370811  1.612432  24.513514
4  23.908333   8.019444  7.530556  1.

<IPython.core.display.Javascript object>

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [8]:
# answer goes here





<IPython.core.display.Javascript object>